# Introduction

### SFRCP: Data Analysis

This is the program that was used for my study, "Comparison of Star Formation Rate in Spiral versus Elliptical Galaxies."

It will be split into different sections for simplicity, with comments explaining each section of code.

##### Note: This is a new program that was created after I decided to revise my original study.

# Imports and Setup

In [ ]:
import csv
import numpy as np
import pandas as pd
from bioinfokit.analys import stat

res = stat()

prefix = 'saved raw data/' # might not need lots of these, check when done
FUV = 'FUV/'
NUV = 'NUV/'
w1 = 'w1/'
w3 = 'w3/'

# Create "Super Galaxies"

In [ ]:
# create super_elliptical and super_spiral (avg all data to get 50 final data points)
# top down design time

In [41]:
sub_sec = 50 # should be the same as sub_sec in MEASURE SFR.ipynb

super_w1_elliptical_mean = np.zeros(sub_sec)
super_w1_spiral_mean = np.zeros(sub_sec)
super_w3_elliptical_mean = np.zeros(sub_sec)
super_w3_spiral_mean = np.zeros(sub_sec)
super_w1_elliptical_sum = np.zeros(sub_sec)
super_w1_spiral_sum = np.zeros(sub_sec)
super_w3_elliptical_sum = np.zeros(sub_sec)
super_w3_spiral_sum = np.zeros(sub_sec)

galaxies_list = [
    'NGC 300',
    'M 99',
    'NGC 3938',
    'NGC 3184',
    'M 100', 
    'M 74',
    'NGC 1304',
    'NGC 3265',
    'NGC 855',
    'NGC 1404', 
    'NGC 584',
    'NGC 5866'
]

with open('GALAXY INFO.csv') as csvfile: galaxy_info = list(csv.reader(csvfile))

raw_data_list = np.empty(len(galaxies_list * 2), dtype=np.dtype('U35'))

for i in range(len(raw_data_list)):   
    if i < 12:
        if galaxies_list[i] == galaxy_info[i + 1][0]:
            gal_type = galaxy_info[i + 1][3]
            mean_gal = galaxies_list[i] + '-raw_mean.csv'
            
            if gal_type == 'Spiral': raw_data_list[i] = 'spiral/' + mean_gal            
            if gal_type == 'Elliptical': raw_data_list[i] = 'elliptical/' + mean_gal
    else:
        i2 =  i - 12
        if galaxies_list[i2] == galaxy_info[i2 + 1][0]:
            gal_type = galaxy_info[i2 + 1][3]
            sum_gal = galaxies_list[i2] + '-raw_sum.csv'

            if gal_type == 'Spiral': raw_data_list[i] = 'spiral/' + sum_gal
            if gal_type == 'Elliptical': raw_data_list[i] = 'elliptical/' + sum_gal

w1_raw_data_list = [w1 + sub for sub in raw_data_list]
w3_raw_data_list = [w3 + sub for sub in raw_data_list]
FUV_raw_data_list = [FUV + sub for sub in raw_data_list]
NUV_raw_data_list = [NUV + sub for sub in raw_data_list]

wise_raw_data_list = w1_raw_data_list + w3_raw_data_list
galex_raw_data_list = FUV_raw_data_list + NUV_raw_data_list

full_raw_data_list = wise_raw_data_list + galex_raw_data_list

print(wise_raw_data_list)

['w1/spiral/NGC 300-raw_mean.csv', 'w1/spiral/M 99-raw_mean.csv', 'w1/spiral/NGC 3938-raw_mean.csv', 'w1/spiral/NGC 3184-raw_mean.csv', 'w1/spiral/M 100-raw_mean.csv', 'w1/spiral/M 74-raw_mean.csv', 'w1/elliptical/NGC 1304-raw_mean.csv', 'w1/elliptical/NGC 3265-raw_mean.csv', 'w1/elliptical/NGC 855-raw_mean.csv', 'w1/elliptical/NGC 1404-raw_mean.csv', 'w1/elliptical/NGC 584-raw_mean.csv', 'w1/elliptical/NGC 5866-raw_mean.csv', 'w1/spiral/NGC 300-raw_sum.csv', 'w1/spiral/M 99-raw_sum.csv', 'w1/spiral/NGC 3938-raw_sum.csv', 'w1/spiral/NGC 3184-raw_sum.csv', 'w1/spiral/M 100-raw_sum.csv', 'w1/spiral/M 74-raw_sum.csv', 'w1/elliptical/NGC 1304-raw_sum.csv', 'w1/elliptical/NGC 3265-raw_sum.csv', 'w1/elliptical/NGC 855-raw_sum.csv', 'w1/elliptical/NGC 1404-raw_sum.csv', 'w1/elliptical/NGC 584-raw_sum.csv', 'w1/elliptical/NGC 5866-raw_sum.csv', 'w3/spiral/NGC 300-raw_mean.csv', 'w3/spiral/M 99-raw_mean.csv', 'w3/spiral/NGC 3938-raw_mean.csv', 'w3/spiral/NGC 3184-raw_mean.csv', 'w3/spiral/M 100

In [43]:
for loop, current_csv in enumerate(wise_raw_data_list):
    end = len(current_csv) - 4
    annoying_csv = current_csv[3:end]
    # sorry for calling it annoying, but i was annoyed with how to deal with this

    galaxy_type = annoying_csv[0:3] # check for 'ell' or else ('spi')
    raw_data_type = annoying_csv[len(annoying_csv) - 4:len(annoying_csv)] # check for 'mean' or else ('_sum')
 
    if galaxy_type == 'ell':
        if raw_data_type == 'mean':
            pass

            # AHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH

        else:
            pass
    else:
        pass

print(super_w1_elliptical_mean)

#repeat above code with w3-mean, w1-sum, w3-sum

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]


# Two Sample T-Tests (with Equal Variances)

In [ ]:
df = pd.read_csv('saved super galaxies/w1-mean-super_galaxies.csv')
res.ttest(df=df, xfac='Galaxy Type', res='SFR', test_type=2)

print(res.summary)

In [ ]:
# repeat above code for w1/w3 and mean/sum

### All `super_galaxies.csv` files should look like this

Galaxy Type|SFR
:---:|:---:
Elliptical|7
Elliptical|continue
Elliptical|for
Elliptical|all
Elliptical|50
Elliptical|data points
Spiral|7
Spiral|continue
Spiral|for
Spiral|all
Spiral|50
Spiral|data points